In [1]:
!pip install numpy SimpleITK plyfile scipy opencv-python 
!apt-get update
!apt-get install -y xvfb libgl1-mesa-glx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 30.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8,066 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [308 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [18.8 kB]
Fetched 8,580 kB in 1s (7,611 kB/s)




The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libfontenc1 libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0
  libglx-mesa0 libglx0

In [3]:
def create_mhd_file(filename, file_id):
    """
    Create a .mhd metadata file for a given .raw volume file
    :param filename: the name of the .raw file
    :param file_id: the numerical id used in the .mhd file name
    """
    mhd_content = f"""ObjectType = Image
        NDims = 3
        BinaryData = True
        BinaryDataByteOrderMSB = False
        CompressedData = False
        TransformMatrix = 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0
        Offset = 0.0 0.0 0.0
        CenterOfRotation = 0.0 0.0 0.0
        AnatomicalOrientation = RAI
        ElementSpacing = 0.49479 0.49479 0.3125
        DimSize = 768 768 1280
        ElementType = MET_USHORT
        ElementDataFile = {filename}"""
    
    # Open and write the mhd content to the new file
    with open(f"scan_{file_id:03}.mhd", 'w') as f:
        f.write(mhd_content)
    print(f"Created scan_{file_id:03}.mhd for raw file {filename}")

# Create the .mhd file for given raw files
file_id = 20
raw_file_name = f"scan_020.raw"
create_mhd_file(raw_file_name, file_id)

Created scan_020.mhd for raw file scan_020.raw


In [4]:
# Loading the volume (raw file) through the generated mhd file above
# right now just loading a single file (scan_001)
import SimpleITK as sitk
import numpy as np

def load_volume(volume_id: int) -> np.ndarray:
    """
    Load a single ultrasound volume from .mhd file
    Args:
        volume_id: ID number of the volume to load (1-5 for labeled data)
    Returns:
        3D numpy array containing ultrasound data
    """
    filename = f"scan_{volume_id:03d}.mhd"
    
    try:
        # Read the .mhd file using SimpleITK
        header = sitk.ReadImage(filename)
        
        # Convert SimpleITK image to numpy array
        volume = sitk.GetArrayFromImage(header)
        
        # Print volume shape and info to verify dimensions
        print(f"Loaded volume shape: {volume.shape}")
        print(f"Data type: {volume.dtype}")
        print(f"Value range: [{volume.min()}, {volume.max()}]")
        
        return volume
        
    except Exception as e:
        print(f"Error loading volume: {e}")
        return None

# 1280 slices, each 768*768 picels
# current value range 0-38685, we need to normalize to 0-1
volume_data = load_volume(file_id)

Loaded volume shape: (1280, 768, 768)
Data type: uint16
Value range: [0, 39289]


In [5]:
# Resized each slice from 768*768 to 256*256 because raw file too big (what they did in the report)
# Converted to float32 and normalized value range to 0-1

import numpy as np
from scipy.ndimage import zoom


def preprocess_volume(volume: np.ndarray, target_size: tuple = (256, 256)) -> np.ndarray:
    
    """
    Preprocess the volume by downsampling and normalizing using scipy
    Args:
        volume: Input volume of shape (1280, 768, 768)
        target_size: Desired size for each slice (width, height)
    Returns:
        Preprocessed volume
    """

    # Calculate zoom factors
    zoom_factors = (1,  # Don't change number of slices
                   target_size[0] / volume.shape[1],  # height scale
                   target_size[1] / volume.shape[2])  # width scale
    
    print("Resizing volume...")
    # Resize the volume using scipy zoom
    processed_volume = zoom(volume, zoom_factors, order=1)
    
    print("Normalizing values...")
    # Normalize to [0,1]
    processed_volume = processed_volume.astype(np.float32)  # Convert to float32 first
    processed_volume = (processed_volume - processed_volume.min()) / (processed_volume.max() - processed_volume.min())
    
    # Print info about processed volume
    print(f"Processed volume shape: {processed_volume.shape}")
    print(f"Processed data type: {processed_volume.dtype}")
    print(f"Processed value range: [{processed_volume.min()}, {processed_volume.max()}]")
    
    return processed_volume


# Example usage:
if __name__ == "__main__":
    # Assuming we have loaded volume_data from previous step
    processed_volume = preprocess_volume(volume_data)

Resizing volume...
Normalizing values...
Processed volume shape: (1280, 256, 256)
Processed data type: float32
Processed value range: [0.0, 1.0]


In [6]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the model
model = load_model('trained_model_reduced.h5')

# Load the processed volume
processed_volume = np.load('train_x/scan_002_processed_volume.npy')

# Check the shape of processed_volume
print(f"Original processed_volume shape: {processed_volume.shape}")

# Reshape it assuming the first dimension is the number of slices
# We'll take each slice (256x256) as a separate example and add a channel dimension
num_slices = processed_volume.shape[0]
processed_volume = processed_volume.reshape((num_slices, 256, 256, 1))

# Predict using the model
prediction = model.predict(processed_volume)
prediction

2024-11-15 20:02:21.305729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 20:02:21.431269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-15 20:02:21.431301: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-15 20:02:21.451649: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 20:02:22.010935: W tensorflow/stream_executor/platform/de

array([[[[1.5897101e-02],
         [2.9124154e-03],
         [6.8757306e-03],
         ...,
         [1.0982599e-05],
         [6.2897854e-04],
         [1.1549268e-03]],

        [[1.2175118e-03],
         [2.3484475e-03],
         [5.5654038e-04],
         ...,
         [2.8881870e-04],
         [8.4619399e-04],
         [1.6496885e-04]],

        [[1.9376396e-03],
         [6.7243673e-04],
         [7.8801510e-05],
         ...,
         [8.3255972e-12],
         [2.2951521e-07],
         [7.5597978e-05]],

        ...,

        [[6.1750456e-05],
         [1.5583842e-07],
         [2.8107861e-08],
         ...,
         [9.9253328e-10],
         [5.7497323e-11],
         [1.3934671e-07]],

        [[3.6878348e-04],
         [3.8693150e-04],
         [7.7683356e-04],
         ...,
         [1.1353395e-12],
         [5.4483365e-08],
         [8.3977202e-06]],

        [[4.2425925e-03],
         [2.2586503e-06],
         [2.8748319e-03],
         ...,
         [1.6879403e-13],
        

It seems there is an issue with loading the `my_model.keras` file. The error indicates that the file might be corrupted or not in the expected format. Let's verify the file format and try again.

In [7]:
from plyfile import PlyElement, PlyData 
# The prediction array has an extra dimension for the channel, which we need to remove.
# We will reshape the prediction array to remove the channel dimension.

# Remove the channel dimension
prediction_reshaped = prediction.reshape((1280, 256, 256))

def numpy_to_ply(prediction, filename='output.ply'):
    # Flatten the prediction array and get all indices
    indices = np.argwhere(prediction)  # Get all indices where prediction is non-zero
    
    # Create a list of vertices
    vertices = [(x, y, z) for x, y, z in indices]
    
    # Define the PLY vertex element
    vertex = np.array(vertices, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex')
    
    # Write to a PLY file
    PlyData([el], text=True).write(filename)

# Convert the reshaped prediction to PLY
numpy_to_ply(prediction_reshaped, 'prediction_output.ply')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract all indices from prediction_reshaped
indices = np.argwhere(prediction_reshaped)

# Plot the points
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(indices[:, 0], indices[:, 1], indices[:, 2], c='b', marker='o')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Scatter Plot of Prediction Points')

plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ee6345b0-5b50-4d75-aba3-45baad6d02c9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>